### What are Hyperparameters?

Parameters = learned from data (e.g., coefficients in Linear Regression, weights in Neural Networks).

Hyperparameters = set before training, they control how the model learns.

Example:

In KNN, k (number of neighbors) is a hyperparameter.

In Decision Trees, max_depth and min_samples_split are hyperparameters.

In SVM, C (margin flexibility) and kernel are hyperparameters.

### Why Tune Hyperparameters?
Default values may not give the best performance.

Right hyperparameters = better accuracy, less overfitting, better generalization.

In [1]:
from sklearn.tree import DecisionTreeClassifier

#### Manual tuning

Instead of blindly using GridSearch, let’s try values manually and compare results.

```py
for depth in [1, 2, 3, 5, 10, None]:
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"max_depth={depth}, Accuracy={accuracy_score(y_test, y_pred):.2f}")
```

#### Methods in hyperparameter tuning
- Manual tuning
- GridSearchCV
- RandomizedSearchCV
- Bayesian optimization

## Grid Search (GridSearchCV)
What: Tries all possible combinations of hyperparameters in a grid.
Good for: Small search space, when you want the exact best parameters.
Bad for: Large datasets, many parameters → very slow.

Example:
If testing max_depth=[3,5,7] and min_samples_split=[2,5] → total = 3 × 2 = 6 models trained.
- first model -> max_depth = 3 and min_samples_split=2 
- Second model -> max_depth = 3 and min_samples_split=5 
- Third model -> max_depth = 5 and min_samples_split=2 
- Fourth model -> max_depth = 5 and min_samples_split=5 
- Fivth model -> max_depth = 7 and min_samples_split=2 
- Sixth model -> max_depth = 7 and min_samples_split=5 

In [2]:
import pandas as pd
data = {
    "Age": [22, 25, 30, 35, 40, 50, 55],
    "Income_LPA": [8, 10, 15, 12, 25, 20, 35],
    "Buys_House": [0, 0, 0, 1, 1, 0, 1]  # 0 = No, 1 = Yes
}

df = pd.DataFrame(data)
df

,Age,Income_LPA,Buys_House
0,22,8,0
1,25,10,0
2,30,15,0
3,35,12,1
4,40,25,1
5,50,20,0
6,55,35,1


In [3]:
from sklearn.model_selection import train_test_split
X = df[["Age", "Income_LPA"]]
y = df["Buys_House"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    "max_depth": [1, 2, 3, None],
    "min_samples_split": [2, 3, 4],
    "random_state": [42,100]
}

# GridSearch
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3)
grid.fit(X_train, y_train)

print("Best Params (GridSearch):", grid.best_params_)
print("Best Score (GridSearch):", grid.best_score_)

d:\A Code\ML-Playground\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best Params (GridSearch): {'max_depth': 1, 'min_samples_split': 2, 'random_state': 42}
Best Score (GridSearch): 0.6666666666666666


## Random Search (RandomizedSearchCV)
- What: Instead of trying all combinations, it tries random subsets.

- Good for: Large search spaces, faster than GridSearch.

- Bad for: Might miss the exact best combo (but usually finds “good enough”).

In [19]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "max_depth": [1, 2, 3, None],
    "min_samples_split": [2, 3, 4, 5, 6]
}

random_search = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=5,   # number of random combinations to try
    cv=3,
    random_state=42
)

random_search.fit(X_train, y_train)

print("Best Params (RandomSearch):", random_search.best_params_)
print("Best Score (RandomSearch):", random_search.best_score_)


Best Params (RandomSearch): {'min_samples_split': 2, 'max_depth': 1}
Best Score (RandomSearch): 0.6666666666666666


d:\A Code\ML-Playground\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


## Bayesian Optimization
- What: Uses probability to guess which hyperparameter region will perform best, and searches there.

- Good for: Expensive models (like deep learning), fewer trials needed.

- Bad for: More complex to implement (libraries like scikit-optimize, optuna).

In [17]:
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    max_depth = trial.suggest_categorical("max_depth", [1, 2, 3, None])
    min_samples_split = trial.suggest_int("min_samples_split", 2, 6)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Run Bayesian Optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best Params (Bayesian):", study.best_params)
print("Best Score (Bayesian):", study.best_value)


d:\A Code\ML-Playground\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-08-30 20:50:34,748] A new study created in memory with name: no-name-519b2551-8dc4-4104-be0f-b042fb92e1ac
[I 2025-08-30 20:50:34,756] Trial 0 finished with value: 0.0 and parameters: {'max_depth': None, 'min_samples_split': 6}. Best is trial 0 with value: 0.0.
[I 2025-08-30 20:50:34,762] Trial 1 finished with value: 1.0 and parameters: {'max_depth': 1, 'min_samples_split': 5}. Best is trial 1 with value: 1.0.
[I 2025-08-30 20:50:34,769] Trial 2 finished with value: 0.0 and parameters: {'max_depth': 3, 'min_samples_split': 3}. Best is trial 1 with value: 1.0.
[I 2025-08-30 20:50:34,775] Trial 3 finished with value: 0.0 and parameters: {'max_depth': None, 'min_samples_split': 3}. Best is trial 1 with value: 1.0.
[I 2025-08-30 20:50

Best Params (Bayesian): {'max_depth': 1, 'min_samples_split': 5}
Best Score (Bayesian): 1.0


In [16]:
!pip install optuna

  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 3.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 3.1 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.1 MB/s  0:00:00
Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl (15 kB)
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)

   ---- ----------------------------------- 1/9 [PyYAML]
   ---- ----------------------------------- 1/9 [PyYAML]
   ---- ----------------------------------- 1/9 [PyYAML]
   ------------- -------------------------- 3/9 [greenlet]
   ------------- -------------------------- 3/9 [greenlet]
   ------------- -------------------------- 3/9 [greenlet]
   -----